In [85]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [105]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
import re
from sklearn.feature_extraction.text import CountVectorizer ,TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score,train_test_split
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [150]:
df_test=pd.read_csv("/content/drive/My Drive/Colab Notebooks/avp/test_oJQbWVk.csv")
df_train=pd.read_csv('/content/drive/My Drive/Colab Notebooks/avp/train_2kmZucJ.csv')
df_train.head()


,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [0]:
stop_words=set(stopwords.words("english"))
stop_words=[x.lower() for x in stop_words]
lemma=WordNetLemmatizer()

shortcuts = {'u': 'you', 'y': 'why', 'r': 'are', 'doin': 'doing', 'hw': 'how', 'k': 'okay', 'm': 'am', 'b4': 'before',
            'idc': "i do not care", 'ty': 'thankyou', 'wlcm': 'welcome', 'bc': 'because', '<3': 'love', 'xoxo': 'love',
            'ttyl': 'talk to you later', 'gr8': 'great', 'bday': 'birthday', 'awsm': 'awesome', 'gud': 'good', 'h8': 'hate',
            'lv': 'love', 'dm': 'direct message', 'rt': 'retweet', 'wtf': 'hate', 'idgaf': 'hate',
             'irl': 'in real life', 'yolo': 'you only live once'}


In [0]:
def clean(text):
    text=text.lower() #conversion to lower
    text=re.sub(r'http\S+', '', text)
    text=re.sub('\W+',' ',text) #extracting only words contain alphabets

    text=word_tokenize(text)# break into words

    emp=[]

    for word in text:
        if word in shortcuts.keys():# removing shoertcuts
            word=shortcuts[word]
        emp.append(word)

    text=emp

    text=[re.sub('\d+','',word) for word in text] #replacing integer with nothing
    text=[ word for word in text if len(word)>2] #removing words of lngth <=2
    text=[word for word in text if word not in stop_words] #remove stopwards
    text=' '.join(lemma.lemmatize(word) for word in text)  #convertig again list of words to sentence
    text=text.replace('  '," ")
    return text



In [0]:
X_train=df_train["tweet"].apply(lambda x:clean(x))
X_test=df_test["tweet"].apply(lambda x:clean(x))
y_train=df_train['label']


In [0]:
vect=TfidfVectorizer(min_df=5).fit(X_train)
X_train_vect=vect.transform(X_train)
X_test_vect=vect.transform(X_test)

In [0]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train_vect, y_train)
pred=model.predict(X_test_vect)
pred=pd.Series(pred)
df=pd.DataFrame()
df["label"]=pred
df["id"]=df_test["id"]
df.to_csv("/content/drive/My Drive/Colab Notebooks/avp/subbmission.csv",index=False)